> # Week 3 Assignment
> ### Scraping Postcode data from wikipedia

In [1]:
import pandas as pd

In [2]:
df_tor = pd.read_html(io="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=0, na_values=['Not assigned'])[0]

#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df_tor.Neighbourhood.fillna(df_tor.Borough, inplace=True)
#ignore/remove dataframe rows where Borough = 'Not assigned'
df_tor.dropna(inplace=True)
df_tor.shape


(212, 3)

In [3]:
#When Postcode is listed twice and has two neighborhoods, combine Neighbourhood values into one row with the neighborhoods separated by a comma
df_etl = df_tor.groupby(['Postcode','Borough'],as_index=False).agg(lambda s: ', '.join(s))

In [4]:
df_etl.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [5]:
df_etl.shape

(103, 3)

### Get the Latitude,Longitude  data for Postcodes

In [6]:
#import lat long data from csv file
latlong = pd.read_csv('http://cocl.us/Geospatial_data')
#uncomment below to check dataframe format 
#latlong.head(10)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [7]:
#join dataframes
df_etl = df_etl.join(latlong.set_index('Postal Code'), on='Postcode')
df_etl.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [8]:
#confirm the dataframe shape still contains the 103 rows
df_etl.shape

(103, 5)

### Use geopy library to get the latitude and longitude values of Toronto.

In [13]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
address = 'Toronto'
geolocator = Nominatim(user_agent="JAC")
location = geolocator.geocode(address)
lat = location.latitude
lon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(lat, lon))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Cluster Toronto Neighbourhoods and load Folium library

In [14]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  42.42 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  27.18 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  26.21 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  24.66 MB/s
Libraries imported.


In [18]:
# create map of Toronto postcodes using latitude and longitude values
map_toronto = folium.Map(location=[lat, lon], zoom_start=10)

# add markers to map
for lat, lon, borough, neighbourhood in zip(df_etl['Latitude'], df_etl['Longitude'], df_etl['Borough'], df_etl['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Let's group postcodes into borough clusters

In [29]:
from folium import plugins

# let's start again with a clean copy of the map of San Francisco
map_toronto = folium.Map(location = [lat, lon], zoom_start = 10)

# instantiate a marker cluster object for the postcodes in the dataframe
postcodes = plugins.MarkerCluster().add_to(map_toronto)

# loop through the dataframe and add each data point to the mark cluster
for lat, lon, label, in zip(df_etl.Latitude, df_etl.Longitude, df_etl.Postcode):
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color='green', icon='ok-sign'),
        popup='lat-long:{}<br>postcode:{}'.format(location[1], location[0]),
    ).add_to(postcodes)

# display map
map_toronto